# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'08-21-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'08-21-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head(20)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-08-22 04:27:49,33.939110,67.709953,37894,1385,28016,8493.0,Afghanistan,97.342962,3.654932
1,NaN,NaN,NaN,Albania,2020-08-22 04:27:49,41.153300,20.168300,8119,240,4096,3783.0,Albania,282.125235,2.956029
2,NaN,NaN,NaN,Algeria,2020-08-22 04:27:49,28.033900,1.659600,40667,1418,28587,10662.0,Algeria,92.738957,3.486857
3,NaN,NaN,NaN,Andorra,2020-08-22 04:27:49,42.506300,1.521800,1045,53,875,117.0,Andorra,1352.488190,5.071770
4,NaN,NaN,NaN,Angola,2020-08-22 04:27:49,-11.202700,17.873900,2068,94,804,1170.0,Angola,6.292166,4.545455
5,NaN,NaN,NaN,Antigua and Barbuda,2020-08-22 04:27:49,17.060800,-61.796400,94,3,89,2.0,Antigua and Barbuda,95.988890,3.191489
6,NaN,NaN,NaN,Argentina,2020-08-22 04:27:49,-38.416100,-63.616700,329043,6730,239806,82507.0,Argentina,728.039259,2.045325
7,NaN,NaN,NaN,Armenia,2020-08-22 04:27:49,40.069100,45.038200,42477,842,35693,5942.0,Armenia,1433.467624,1.982249
8,NaN,NaN,Australian Capital Territory,Australia,2020-08-22 04:27:49,-35.473500,149.012400,113,3,110,0.0,"Australian Capital Territory, Australia",26.395702,2.654867
9,NaN,NaN,New South Wales,Australia,2020-08-22 04:27:49,-33.868800,151.209300,3981,52,2994,935.0,"New South Wales, Australia",49.039172,1.306204


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,37345,37424,37431,37551,37596,37599,37599,37599,37856,37894
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,6817,6971,7117,7260,7380,7499,7654,7812,7967,8119
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,36699,37187,37664,38133,38583,39025,39444,39847,40258,40667


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1354,1363,1363,1370,1375,1375,1375,1375,1385,1385
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,208,213,219,225,228,230,232,234,238,240
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1333,1341,1351,1360,1370,1379,1391,1402,1411,1418


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,26694,26714,26714,27166,27166,27166,27166,27166,27681,28016
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3552,3616,3695,3746,3794,3816,3871,3928,3986,4096
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,25627,26004,26308,26644,27017,27347,27653,27971,28281,28587


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1188,1196,1196,1206,1218,1219,1235,1241,1240,1255
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,3710,3744,3791,3839,3877,3888,3906,3931,3957,3997
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,581,586,590,598,604,605,606,613,619,622


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,22,22,22,22,22,22,22,22,22,22
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,29,29,29,29,29,29,30,30,31,32
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,6,6,6,6,6,6,6,7,7,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,1001.0,Autauga,Alabama,US,2020-08-22 04:27:49,32.539527,-86.644082,1255,22,0,1233.0,"Autauga, Alabama, US",2246.326227,1.752988
668,1075.0,Lamar,Alabama,US,2020-08-22 04:27:49,33.779950,-88.096680,259,2,0,257.0,"Lamar, Alabama, US",1876.131836,0.772201
669,1077.0,Lauderdale,Alabama,US,2020-08-22 04:27:49,34.901719,-87.656247,1308,22,0,1286.0,"Lauderdale, Alabama, US",1410.561960,1.681957


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,5622540,175363,1965056
Brazil,3532330,113358,2855558
India,2975701,55794,2222577


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,5622540,175363,1965056,3482121,2020-08-22 04:27:49,37.919464,-91.363237
Brazil,3532330,113358,2855558,563414,2020-08-22 04:27:49,-12.669522,-48.480493
India,2975701,55794,2222577,697330,2020-08-22 04:27:49,23.088275,81.806127
Russia,944671,16148,759639,168884,2020-08-22 04:27:49,54.546312,62.120860
South Africa,603338,12843,500102,90393,2020-08-22 04:27:49,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,658900,11946,0
Florida,593286,10168,0
Texas,585632,11418,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,658900,11946,0,646954,2020-08-22 04:27:49,37.843962,-120.728594
Florida,593286,10168,0,583118,2020-08-22 04:27:49,28.940755,-82.700744
Texas,585632,11418,0,574214,2020-08-22 04:27:49,31.660643,-98.653069
New York,428512,32864,0,395648,2020-08-22 04:27:49,42.671593,-75.579694
Georgia,249630,4998,0,244632,2020-08-22 04:27:49,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,231574,23641,0
California,Los Angeles,229054,5491,0
Florida,Miami-Dade,150305,2219,0
Arizona,Maricopa,131166,2743,0
Illinois,Cook,119644,4997,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,231574,23641,0,207933,2020-08-22 04:27:49,40.767273,-73.971526,36061.0
California,Los Angeles,229054,5491,0,223563,2020-08-22 04:27:49,34.308284,-118.228241,6037.0
Florida,Miami-Dade,150305,2219,0,148086,2020-08-22 04:27:49,25.611236,-80.551706,12086.0
Arizona,Maricopa,131166,2743,0,128423,2020-08-22 04:27:49,33.348359,-112.491815,4013.0
Illinois,Cook,119644,4997,0,114647,2020-08-22 04:27:49,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-19,37599,7812,39847,1024,2015,94,312659,42056,24236,24084,...,323008,1493,37112,36868,994,17606,10,1892,10218,5643
2020-08-20,37856,7967,40258,1024,2044,94,320884,42319,24407,24431,...,324203,1506,37547,37567,1007,17989,10,1899,10372,5745
2020-08-21,37894,8119,40667,1045,2068,94,329043,42477,24602,24762,...,325263,1516,38074,38219,1009,18313,10,1906,10627,5815


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-19,1375,234,1402,53,92,3,6330,833,463,729,...,41483,40,248,303,25,119,1,539,269,150
2020-08-20,1385,238,1411,53,93,3,6517,836,472,729,...,41489,41,252,311,25,120,1,541,274,151
2020-08-21,1385,240,1418,53,94,3,6730,842,485,730,...,41491,42,260,317,25,122,1,542,277,152


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-19,27166,3928,27971,875,698,88,228725,35226,15247,20958,...,1495,1228,32944,25416,533,10312,8,1055,9126,4442
2020-08-20,27681,3986,28281,875,742,89,233651,35476,17852,21093,...,1544,1242,33261,26330,542,10682,8,1058,9126,4525
2020-08-21,28016,4096,28587,875,804,89,239806,35693,18458,21260,...,1544,1249,33989,27306,545,11102,8,1058,9126,4587


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1188,1196,1196,1206,1218,1219,1235,1241,1240,1255
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,230,237,237,239,240,245,247,249,255,259
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1203,1213,1219,1231,1238,1240,1269,1278,1287,1308
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,356,360,364,367,370,370,377,379,383,389
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,2724,2736,2735,2760,2776,2779,2861,2888,2899,2908


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-19,111478,4438,0,195557,54216,647274,53882,51314,16643,49,...,137800,574633,47521,1533,828,109019,68689,8807,67493,3431
2020-08-20,112449,4520,0,196280,54765,652267,54211,51432,16718,49,...,139175,579892,47982,1537,869,109879,69389,8987,68233,3457
2020-08-21,113632,4588,0,196899,55652,658900,54211,51519,16770,49,...,140844,585632,48445,1541,932,110857,69779,9077,69059,3524


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-19        1241    3931     613  490    901     522    796    2028   
2020-08-20        1240    3957     619  494    912     527    796    2072   
2020-08-21        1255    3997     622  501    930     528    802    2105   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-19          859      313  ...     152      7      101       41   
2020-08-20          861      319  ...     153      7      108       41   
2020-08-21          865      324  ...     156      7      112       44   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-19            285   394   283          0      104     12  
2020-08-20            285   398   283          0      106      0  
2020-08-21            288   399   283          0      106     11  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-19,1944,29,0,4634,631,11690,1900,4457,595,0,...,1452,10934,377,58,9,2410,1822,166,1060,34
2020-08-20,1974,29,0,4684,641,11810,1903,4458,595,0,...,1488,11174,381,58,10,2427,1837,167,1067,34
2020-08-21,1996,30,0,4688,663,11946,1903,4460,600,0,...,1549,11418,383,58,10,2436,1850,171,1068,37


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-19          22      30       7    5      5      13     36      23   
2020-08-20          22      31       7    6      5      13     36      23   
2020-08-21          22      32       7    6      6      13     36      23   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-19           38        8  ...       0      0        0        0   
2020-08-20           38        9  ...       0      0        0        0   
2020-08-21           38        9  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-19              0     0     0         33        0      0  
2020-08-20              0     0     0         33        0      0  
2020-08-21              0     0     0         36        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-19,0.000000,0.020643,0.010217,0.018905,0.024924,0.010753,0.021875,0.005018,0.010296,0.010701,...,0.002579,0.005387,0.020907,0.032804,0.005056,0.017335,0.0,0.001588,0.023745,0.049275
2020-08-20,0.006835,0.019841,0.010314,0.000000,0.014392,0.000000,0.026307,0.006254,0.007056,0.014408,...,0.003700,0.008707,0.011721,0.018960,0.013078,0.021754,0.0,0.003700,0.015071,0.018075
2020-08-21,0.001004,0.019079,0.010159,0.020508,0.011742,0.000000,0.025427,0.003734,0.007990,0.013548,...,0.003270,0.006640,0.014036,0.017356,0.001986,0.018011,0.0,0.003686,0.024585,0.012185


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-19,0.000000,0.008621,0.007908,0.0,0.022222,0.0,0.046627,0.001202,0.028889,0.000000,...,0.000410,0.00000,0.024793,0.020202,-0.038462,0.053097,0.0,0.003724,0.018939,0.063830
2020-08-20,0.007273,0.017094,0.006419,0.0,0.010870,0.0,0.029542,0.003601,0.019438,0.000000,...,0.000145,0.02500,0.016129,0.026403,0.000000,0.008403,0.0,0.003711,0.018587,0.006667
2020-08-21,0.000000,0.008403,0.004961,0.0,0.010753,0.0,0.032684,0.007177,0.027542,0.001372,...,0.000048,0.02439,0.031746,0.019293,0.000000,0.016667,0.0,0.001848,0.010949,0.006623


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-19,0.000000,0.014725,0.011500,0.006904,0.046477,0.000000,0.023236,0.006975,0.021438,0.004217,...,0.001340,0.007383,0.027509,0.034811,0.013308,0.037529,0.0,0.002852,0.039881,0.082095
2020-08-20,0.018958,0.014766,0.011083,0.000000,0.063037,0.011364,0.021537,0.007097,0.170853,0.006441,...,0.032776,0.011401,0.009622,0.035962,0.016886,0.035881,0.0,0.002844,0.000000,0.018685
2020-08-21,0.012102,0.027597,0.010820,0.000000,0.083558,0.000000,0.026343,0.006117,0.033946,0.007917,...,0.000000,0.005636,0.021887,0.037068,0.005535,0.039318,0.0,0.000000,0.000000,0.013702


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-19,0.010121,0.015328,NaN,0.003268,0.013629,0.014313,0.005036,0.001151,0.003013,0.0,...,0.014952,0.010234,0.007719,0.001961,0.040201,0.006806,0.006241,0.007897,0.019593,0.021131
2020-08-20,0.008710,0.018477,NaN,0.003697,0.010126,0.007714,0.006106,0.002300,0.004506,0.0,...,0.009978,0.009152,0.009701,0.002609,0.049517,0.007889,0.010191,0.020438,0.010964,0.007578
2020-08-21,0.010520,0.015044,NaN,0.003154,0.016196,0.010169,0.000000,0.001692,0.003110,0.0,...,0.011992,0.009898,0.009649,0.002602,0.072497,0.008901,0.005620,0.010014,0.012106,0.019381


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-19      0.004858  0.006400  0.011551  0.029412  0.015784  0.035714   
2020-08-20     -0.000806  0.006614  0.009788  0.008163  0.012209  0.009579   
2020-08-21      0.012097  0.010109  0.004847  0.014170  0.019737  0.001898   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-08-19      0.000000  0.023209  0.000000  0.000000  ...  0.027027    0.0   
2020-08-20      0.000000  0.021696  0.002328  0.019169  ...  0.006579    0.0   
2020-08-21      0.007538  0.015927  0.004646  0.015674  ...  0.019608    0.0   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2020-08-19      0.109890  0.025000   0.021505  0.002545  0.017986        NaN   
2020-08-20      0.069307  0.000000   0.000000  0.010152  0.000000        NaN   
2020-08-21      0.037037  0.073171   0.010526  0.002513  0.000000        NaN   

Province_State                   
Admin2          Washakie Weston  
2020-08-19      0.019608    0.5  
2020-08-20      0.019231   -1.0  
2020-08-21      0.000000    inf  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-19,0.004132,0.000000,NaN,0.023184,0.019386,0.016168,0.000527,0.000224,0.003373,NaN,...,0.019663,0.023975,0.021680,0.0,0.000000,0.005843,0.007743,0.024691,0.020212,0.133333
2020-08-20,0.015432,0.000000,NaN,0.010790,0.015848,0.010265,0.001579,0.000224,0.000000,NaN,...,0.024793,0.021950,0.010610,0.0,0.111111,0.007054,0.008233,0.006024,0.006604,0.000000
2020-08-21,0.011145,0.034483,NaN,0.000854,0.034321,0.011516,0.000000,0.000449,0.008403,NaN,...,0.040995,0.021836,0.005249,0.0,0.000000,0.003708,0.007077,0.023952,0.000937,0.088235


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga   Baldwin   Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-19         0.0  0.000000  0.166667  0.0    0.0     0.0    0.0     0.0   
2020-08-20         0.0  0.033333  0.000000  0.2    0.0     0.0    0.0     0.0   
2020-08-21         0.0  0.032258  0.000000  0.0    0.2     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-19          0.0    0.000  ...     NaN    NaN      NaN      NaN   
2020-08-20          0.0    0.125  ...     NaN    NaN      NaN      NaN   
2020-08-21          0.0    0.000  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-19            NaN   NaN   NaN   0.137931      NaN    NaN  
2020-08-20            NaN   NaN   NaN   0.000000      NaN    NaN  
2020-08-21            NaN   NaN   NaN   0.090909      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-19,0.000222,0.019833,0.010766,0.011701,0.020454,0.005546,0.021308,0.004076,0.010075,0.010594,...,0.002884,0.009814,0.019010,0.031310,0.008816,0.020646,1.927471e-19,0.003122,0.024722,0.031261
2020-08-20,0.003529,0.019837,0.010540,0.005851,0.017423,0.002773,0.023807,0.005165,0.008565,0.012501,...,0.003292,0.009261,0.015366,0.025135,0.010947,0.021200,9.637353e-20,0.003411,0.019897,0.024668
2020-08-21,0.002266,0.019458,0.010350,0.013179,0.014582,0.001387,0.024617,0.004449,0.008277,0.013025,...,0.003281,0.007950,0.014701,0.021245,0.006467,0.019606,4.818676e-20,0.003548,0.022241,0.018426


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-19,0.000507,0.010034,0.007844,0.000075,0.019563,5.923390e-39,0.038185,0.004308,0.032494,0.000320,...,0.000307,0.007002,0.024275,0.024422,0.001798,0.031942,0.0,0.004279,0.012712,0.048417
2020-08-20,0.003890,0.013564,0.007132,0.000038,0.015216,2.961695e-39,0.033864,0.003955,0.025966,0.000160,...,0.000226,0.016001,0.020202,0.025412,0.000899,0.020172,0.0,0.003995,0.015650,0.027542
2020-08-21,0.001945,0.010984,0.006046,0.000019,0.012985,1.480847e-39,0.033274,0.005566,0.026754,0.000766,...,0.000137,0.020196,0.025974,0.022352,0.000449,0.018420,0.0,0.002922,0.013299,0.017082


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-19,0.000601,0.013573,0.011763,0.004516,0.040482,0.005206,0.026858,0.006878,0.024621,0.004436,...,0.001705,0.007959,0.029033,0.036797,0.043878,0.024752,1.129377e-21,0.008362,0.033702,0.167202
2020-08-20,0.009779,0.014170,0.011423,0.002258,0.051759,0.008285,0.024197,0.006988,0.097737,0.005439,...,0.017241,0.009680,0.019328,0.036379,0.030382,0.030316,5.646886e-22,0.005603,0.016851,0.092944
2020-08-21,0.010941,0.020883,0.011121,0.001129,0.067659,0.004142,0.025270,0.006552,0.065841,0.006678,...,0.008620,0.007658,0.020608,0.036724,0.017958,0.034817,2.823443e-22,0.002801,0.008426,0.053323


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-19,0.009951,0.015641,NaN,0.003721,0.010681,0.013195,0.004995,0.001484,0.003829,3.726858e-48,...,0.012043,0.011710,0.006899,0.003122,0.038684,0.007427,0.006831,0.009792,0.012169,0.016477
2020-08-20,0.009331,0.017059,NaN,0.003709,0.010403,0.010455,0.005551,0.001892,0.004168,1.863429e-48,...,0.011011,0.010431,0.008300,0.002866,0.044101,0.007658,0.008511,0.015115,0.011567,0.012027
2020-08-21,0.009925,0.016052,NaN,0.003431,0.013300,0.010312,0.002775,0.001792,0.003639,9.317144e-49,...,0.011501,0.010165,0.008975,0.002734,0.058299,0.008279,0.007066,0.012565,0.011836,0.015704


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-19      0.006866  0.006091  0.007650  0.017828  0.016362  0.022291   
2020-08-20      0.003030  0.006353  0.008719  0.012995  0.014285  0.015935   
2020-08-21      0.007563  0.008231  0.006783  0.013583  0.017011  0.008916   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-08-19      0.001598  0.020070  0.001481  0.009239  ...  0.019490   
2020-08-20      0.000799  0.020883  0.001905  0.014204  ...  0.013035   
2020-08-21      0.004168  0.018405  0.003275  0.014939  ...  0.016321   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-08-19      0.043229  0.077713  0.012601   0.013558  0.004722  0.010185   
2020-08-20      0.021615  0.073510  0.006301   0.006779  0.007437  0.005093   
2020-08-21      0.010807  0.055273  0.039736   0.008653  0.004975  0.002546   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-08-19           -1.0  0.021470  0.280227  
2020-08-20           -1.0  0.020351 -0.359887  
2020-08-21           -1.0  0.010175 -0.359887  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-19,0.005600,0.010246,NaN,0.013927,0.018139,0.014211,0.000888,0.000213,0.001708,NaN,...,0.020280,0.020342,0.015309,1.079307e-06,0.000008,0.004544,0.008376,0.016774,0.010764,0.067011
2020-08-20,0.010516,0.005123,NaN,0.012358,0.016993,0.012238,0.001233,0.000219,0.000854,NaN,...,0.022537,0.021146,0.012960,5.396536e-07,0.055559,0.005799,0.008305,0.011399,0.008684,0.033505
2020-08-21,0.010831,0.019803,NaN,0.006606,0.025657,0.011877,0.000617,0.000334,0.004629,NaN,...,0.031766,0.021491,0.009105,2.698268e-07,0.027780,0.004754,0.007691,0.017676,0.004811,0.060870


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-19      0.000095  0.008965  0.083724  0.000073  0.000570  0.022727   
2020-08-20      0.000047  0.021149  0.041862  0.100037  0.000285  0.011364   
2020-08-21      0.000024  0.026704  0.020931  0.050018  0.100143  0.005682   

Province_State                                              ... Wyoming  \
Admin2            Butler   Calhoun      Chambers  Cherokee  ...    Park   
2020-08-19      0.000007  0.037554  2.329109e-09  0.000140  ...     NaN   
2020-08-20      0.000003  0.018777  1.164555e-09  0.062570  ...     NaN   
2020-08-21      0.000002  0.009389  5.822774e-10  0.031285  ...     NaN   

Province_State                                                             \
Admin2         Platte Sheridan Sublette Sweetwater Teton Uinta Unassigned   
2020-08-19        NaN      NaN      NaN        NaN   NaN   NaN   0.069322   
2020-08-20        NaN      NaN      NaN        NaN   NaN   NaN   0.034661   
2020-08-21        NaN      NaN      NaN        NaN   NaN   NaN   0.062785   

Province_State                  
Admin2         Washakie Weston  
2020-08-19          NaN    NaN  
2020-08-20          NaN    NaN  
2020-08-21          NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200821,AK,5403,316132,NaN,46.0,NaN,NaN,NaN,6.0,...,321535,1,0,ac97da2a3705756521d2ad109926d4e477195879,0,0,0,0,0,NaN
1,20200821,AL,112770,784330,NaN,1168.0,13330.0,NaN,1348.0,NaN,...,897100,16,0,7d05c9893cc3fff06668cf52e6806a369e1f5b63,0,0,0,0,0,NaN
2,20200821,AR,55652,602939,NaN,509.0,3856.0,NaN,NaN,120.0,...,658591,22,66,95be4b99522466d3dd6b1045b544cd190c7e6e06,0,0,0,0,0,NaN
3,20200821,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,ff5ed9b9a63e22b90be5cda6c0101578ae5b8fbe,0,0,0,0,0,NaN
4,20200821,AZ,196899,931089,NaN,1068.0,21210.0,365.0,NaN,234.0,...,1127988,4,67,ae9aeb72f39e0b417a6a845dea6179dffa0a277e,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200821,AK,5403.0,316132.0,NaN,46.0,NaN,NaN,NaN,6.0,...,321535,1,0,ac97da2a3705756521d2ad109926d4e477195879,0,0,0,0,0,NaN
1,20200821,AL,112770.0,784330.0,NaN,1168.0,13330.0,NaN,1348.0,NaN,...,897100,16,0,7d05c9893cc3fff06668cf52e6806a369e1f5b63,0,0,0,0,0,NaN
2,20200821,AR,55652.0,602939.0,NaN,509.0,3856.0,NaN,NaN,120.0,...,658591,22,66,95be4b99522466d3dd6b1045b544cd190c7e6e06,0,0,0,0,0,NaN
3,20200821,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,ff5ed9b9a63e22b90be5cda6c0101578ae5b8fbe,0,0,0,0,0,NaN
4,20200821,AZ,196899.0,931089.0,NaN,1068.0,21210.0,365.0,NaN,234.0,...,1127988,4,67,ae9aeb72f39e0b417a6a845dea6179dffa0a277e,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-21,AK,5403,316132,NaN,46.0,NaN,NaN,NaN,6.0,NaN,...,321535,1,0,ac97da2a3705756521d2ad109926d4e477195879,0,0,0,0,0,NaN
2020-08-21,AL,112770,784330,NaN,1168.0,13330.0,NaN,1348.0,NaN,734.0,...,897100,16,0,7d05c9893cc3fff06668cf52e6806a369e1f5b63,0,0,0,0,0,NaN
2020-08-21,AR,55652,602939,NaN,509.0,3856.0,NaN,NaN,120.0,499.0,...,658591,22,66,95be4b99522466d3dd6b1045b544cd190c7e6e06,0,0,0,0,0,NaN
2020-08-21,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,ff5ed9b9a63e22b90be5cda6c0101578ae5b8fbe,0,0,0,0,0,NaN
2020-08-21,AZ,196899,931089,NaN,1068.0,21210.0,365.0,NaN,234.0,NaN,...,1127988,4,67,ae9aeb72f39e0b417a6a845dea6179dffa0a277e,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-21,AK,5403.0,316132.0,NaN,46.0,NaN,NaN,NaN,6.0,NaN,...,321535,1,0,ac97da2a3705756521d2ad109926d4e477195879,0,0,0,0,0,NaN
2020-08-21,AL,112770.0,784330.0,NaN,1168.0,13330.0,NaN,1348.0,NaN,734.0,...,897100,16,0,7d05c9893cc3fff06668cf52e6806a369e1f5b63,0,0,0,0,0,NaN
2020-08-21,AR,55652.0,602939.0,NaN,509.0,3856.0,NaN,NaN,120.0,499.0,...,658591,22,66,95be4b99522466d3dd6b1045b544cd190c7e6e06,0,0,0,0,0,NaN
2020-08-21,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,ff5ed9b9a63e22b90be5cda6c0101578ae5b8fbe,0,0,0,0,0,NaN
2020-08-21,AZ,196899.0,931089.0,NaN,1068.0,21210.0,365.0,NaN,234.0,NaN,...,1127988,4,67,ae9aeb72f39e0b417a6a845dea6179dffa0a277e,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE